In [2]:
import numpy as np
import pandas as pd
import pyarrow
import ast
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MinMaxScaler, StandardScaler
data_movies = pd.read_parquet("../Source/data_movies.parquet", engine="pyarrow")

In [3]:
data_movies_small = data_movies.head(5000)

In [4]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler

# Paso 1: Procesar el texto de 'overview' usando TF-IDF
tfidf = TfidfVectorizer(stop_words='english')

# Llenar nulos en 'overview' con cadenas vacías
data_movies_small['overview'] = data_movies_small['overview'].fillna('')

# Aplicar TF-IDF a la columna 'overview'
tfidf_matrix = tfidf.fit_transform(data_movies_small['overview'])

# Paso 2: Procesar 'genre_name' usando One-Hot Encoding
data_movies_small['genre_name'] = data_movies_small['genre_name'].fillna('')

# One-Hot Encoding para el género
genre_dummies = pd.get_dummies(data_movies_small['genre_name'])

# Paso 3: Normalizar 'release_year'
scaler = MinMaxScaler()

# Rellenar nulos y escalar la columna de 'release_year'
data_movies_small['release_year'] = data_movies_small['release_year'].fillna(0)
release_year_scaled = scaler.fit_transform(data_movies_small[['release_year']])

# Paso 4: Combinar las características (TF-IDF, género, y año de lanzamiento)
from scipy.sparse import hstack

# Combinar todas las características en una sola matriz
features = hstack([tfidf_matrix, genre_dummies, release_year_scaled])

# Paso 5: Calcular la similitud del coseno
cosine_sim = cosine_similarity(features, features)

# Función para obtener recomendaciones
def get_recommendations(title, cosine_sim=cosine_sim):
    # Verificar si el título existe en el DataFrame
    if title not in data_movies_small['title'].values:
        return f"No se encontró el título '{title}' en el dataset."
    # Obtener el índice de la película que coincide con el título
    idx = data_movies_small[data_movies_small['title'] == title].index[0]
    
    # Obtener las puntuaciones de similitud de coseno para esa película
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # Ordenar las películas en función de las puntuaciones de similitud
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Obtener los índices de las 10 películas más similares
    sim_scores = sim_scores[1:11]
    
    # Obtener los índices de las películas
    movie_indices = [i[0] for i in sim_scores]
    
    # Retornar los títulos de las 10 películas más similares
    return data_movies_small['title'].iloc[movie_indices]

/tmp/ipykernel_14702/4155649624.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_movies_small['overview'] = data_movies_small['overview'].fillna('')
/tmp/ipykernel_14702/4155649624.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_movies_small['genre_name'] = data_movies_small['genre_name'].fillna('')
/tmp/ipykernel_14702/4155649624.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

In [6]:
get_recommendations("Titanic")

1672            Deep Rising
1612    Alien: Resurrection
1635               Scream 2
1210              Bad Taste
1577               Phantoms
1081          The Sleepover
1518          Event Horizon
1288            Blood Beach
778         The Frighteners
421          Body Snatchers
Name: title, dtype: object

In [11]:
def get_recommendations(title):
    # Verificar si el título existe en el DataFrame
    if title not in data_movies_small['title'].values:
        return f"No se encontró el título '{title}' en el dataset."

    # Obtener el índice de la película que coincide con el título
    idx = data_movies_small[data_movies_small['title'] == title].index[0]

    # Obtener las puntuaciones de similitud de coseno para esa película
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Ordenar las películas en función de las puntuaciones de similitud
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Obtener los índices de las 5 películas más similares
    sim_scores = sim_scores[1:6]  # Cambiado para obtener solo las 5 películas más similares

    # Obtener los índices de las películas
    movie_indices = [i[0] for i in sim_scores]

    # Retornar los títulos de las 5 películas más similares
    return data_movies_small['title'].iloc[movie_indices]

In [12]:
get_recommendations("Titanic")

1672            Deep Rising
1612    Alien: Resurrection
1635               Scream 2
1210              Bad Taste
1577               Phantoms
Name: title, dtype: object

Como genre_name es una columna categórica (puede tener múltiples géneros en una sola película), puedes aplicar One-Hot Encoding o MultiLabelBinarizer para convertirla en una matriz binaria.

One-Hot Encoding: Convierte cada género en una columna binaria (0 o 1), indicando si la película pertenece a ese género o no.

2. Escalar release_year:
release_year es una columna numérica, pero sus valores pueden variar mucho (por ejemplo, de 1920 a 2020). Para que esta columna esté en una escala comparable a las otras, necesitas escalar sus valores, por ejemplo, utilizando MinMaxScaler para poner los valores entre 0 y 1.

In [13]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler

# Paso 1: Procesar el texto de 'overview' usando TF-IDF
tfidf = TfidfVectorizer(stop_words='english')

# Llenar nulos en 'overview' con cadenas vacías
data_movies_small['overview'] = data_movies_small['overview'].fillna('')

# Aplicar TF-IDF a la columna 'overview'
tfidf_matrix = tfidf.fit_transform(data_movies_small['overview'])

# Paso 2: Procesar 'genre_name' usando One-Hot Encoding
data_movies_small['genre_name'] = data_movies_small['genre_name'].fillna('')

# One-Hot Encoding para el género
genre_dummies = pd.get_dummies(data_movies_small['genre_name'])

# Paso 3: Normalizar 'release_year'
scaler = MinMaxScaler()

# Rellenar nulos y escalar la columna de 'release_year'
data_movies_small['release_year'] = data_movies_small['release_year'].fillna(0)
release_year_scaled = scaler.fit_transform(data_movies_small[['release_year']])

# Paso 4: Combinar las características (TF-IDF, género, y año de lanzamiento)
from scipy.sparse import hstack

# Combinar todas las características en una sola matriz
features = hstack([tfidf_matrix, genre_dummies, release_year_scaled])

# Paso 5: Calcular la similitud del coseno
cosine_sim = cosine_similarity(features)

/tmp/ipykernel_14702/2524686418.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_movies_small['overview'] = data_movies_small['overview'].fillna('')
/tmp/ipykernel_14702/2524686418.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_movies_small['genre_name'] = data_movies_small['genre_name'].fillna('')
/tmp/ipykernel_14702/2524686418.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

In [14]:
def get_recommendations(title):
    # Verificar si el título existe en el DataFrame
    if title not in data_movies_small['title'].values:
        return f"No se encontró el título '{title}' en el dataset."

    # Obtener el índice de la película que coincide con el título
    idx = data_movies_small[data_movies_small['title'] == title].index[0]

    # Obtener las puntuaciones de similitud de coseno para esa película
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Ordenar las películas en función de las puntuaciones de similitud
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Obtener los índices de las 5 películas más similares
    sim_scores = sim_scores[1:6]  # Cambiado para obtener solo las 5 películas más similares

    # Obtener los índices de las películas
    movie_indices = [i[0] for i in sim_scores]

    # Retornar los títulos de las 5 películas más similares
    return data_movies_small['title'].iloc[movie_indices]

In [16]:
get_recommendations("Titanic")

1672            Deep Rising
1612    Alien: Resurrection
1635               Scream 2
1210              Bad Taste
1577               Phantoms
Name: title, dtype: object